# Problem 1

In [1]:
import numpy as np

data = np.load('mnist.npz')
x_train, y_train = data['x_train'], data['y_train']
x_test, y_test = data['x_test'], data['y_test']

In [2]:
# reshape to vectors, change dtype from int8 to float32 and normalize [0,255] -> [0,1]
x_train = x_train.reshape(x_train.shape[0], -1).astype(np.float32) / 255
x_test = x_test.reshape(x_test.shape[0], -1).astype(np.float32) / 255

In [3]:
# create a validation set
from sklearn import model_selection

x_train, x_valid, y_train, y_valid = model_selection.train_test_split(
    x_train, y_train, test_size=10000, stratify=y_train)

In [4]:
import math

class Classifier(object):
    def __init__(self,
                 hidden_dims=(400, 100),
                 n_hidden=2,
                 mode="train",
                 datapath=None,
                 model_path=None):
        #weights and bias structure
        self.weights, self.bias = [], []
        #layer 1
        self.weights.append(np.empty((hidden_dims[0], 784)))
        self.bias.append(np.zeros((hidden_dims[0])))
        #hidden layer
        for i in range(n_hidden - 1):
            self.weights.append(np.empty((hidden_dims[i + 1], hidden_dims[i])))
            self.bias.append(np.zeros((hidden_dims[i + 1])))
        #output layer
        self.weights.append(np.empty((10, hidden_dims[-1])))
        self.bias.append(np.zeros((10)))

    def initialize_weights(self, method="glorot"):
        for i, w in enumerate(self.weights):
            if method is "glorot":
                d = math.sqrt(6 / (w.shape[0] + w.shape[1]))
                self.weights[i] = np.random.uniform(
                    low=-d, high=d, size=w.shape)
            if method is "normal":
                self.weights[i] = np.random.normal(
                    loc=0, scale=1, size=w.shape)
            if method is "zero":
                self.weights[i] = np.zeros(shape=w.shape)

    def forward(self, input):
        self.cache = [input]
        for w, b in zip(self.weights, self.bias):
            self.cache.append(self.activation(w @ self.cache[-1] + b))
        return self.softmax(self.cache.pop())

    def activation(self, input):
        return np.maximum(0, input)

    def loss(self, prediction, label):
        return -math.log(self.softmax(prediction)[label])

    def softmax(self, input):
        m = np.max(input)
        return np.exp(input - m) / np.sum(np.exp(input - m))

    def backward(self, output, label):
        grad_pre_activation = np.asarray([
            out - 1 if i == label else out for i, out in enumerate(output)
        ]).reshape(-1, 1)
        self.grad_w, self.grad_b = [], []
        # we go from the last layer to the first one
        for i, (w, b) in enumerate(
                zip(reversed(self.weights), reversed(self.bias))):
            previous_hidden_layer = np.asarray(list(reversed(self.cache))[i])
            self.grad_w.insert(
                0,
                grad_pre_activation @ previous_hidden_layer.reshape(-1, 1).T)
            self.grad_b.insert(0, grad_pre_activation.reshape(-1))
            grad_previous_hidden_layer = w.T @ grad_pre_activation
            grad_pre_activation = grad_previous_hidden_layer * np.asarray(
                [1 if x > 0 else 0
                 for x in list(reversed(self.cache))[i]]).reshape(-1, 1)

    def update(self, lr):
        for i, (gw, gb) in enumerate(zip(self.grad_w, self.grad_b)):
            self.weights[i] = self.weights[i] - lr * gw
            self.bias[i] = self.bias[i] - lr * gb

    def train(self, inputs, labels, epochs=1, lr=0.001, verbose=True):
        total_loss = []
        for epoch in range(epochs):
            loss = []
            for i, (x, y) in enumerate(zip(inputs, labels), 1):
                pred = clf.forward(x)
                loss.append(clf.loss(pred, y))
                if verbose and i % 100 == 0:
                    print(
                        "\repoch {:2d}: {:.3f}".format(epoch + 1, np.mean(loss)),
                        end="")
                clf.backward(pred, y)
                clf.update(lr)
            if verbose:
                print("\repoch {:2d}: {:.3f}".format(epoch + 1, np.mean(loss)))
            total_loss.append(np.mean(loss))
        return total_loss

    def test(self, inputs, labels):
        loss, acc = zip(*[(self.loss(self.forward(x), y),
                           np.argmax(self.forward(x)) == y)
                          for x, y in zip(inputs, labels)])
        loss, acc = np.mean(loss), np.mean(acc)
        return np.mean(loss), np.mean(acc)

In [5]:
clf = Classifier()
clf.initialize_weights("zero")
z_loss = clf.train(x_train, y_train, epochs=10, lr=1e-3)

epoch  1: 2.302
epoch  2: 2.302
epoch  3: 2.302
epoch  4: 2.302
epoch  5: 2.302
epoch  6: 2.302
epoch  7: 2.302
epoch  8: 2.302
epoch  9: 2.302
epoch 10: 2.302


In [6]:
clf = Classifier()
clf.initialize_weights("normal")
n_loss = clf.train(x_train, y_train, epochs=10, lr=1e-3)

epoch  1: 1.926
epoch  2: 1.797
epoch  3: 1.727
epoch  4: 1.686
epoch  5: 1.662
epoch  6: 1.647
epoch  7: 1.635
epoch  8: 1.625
epoch  9: 1.619
epoch 10: 1.611


In [7]:
clf = Classifier()
clf.initialize_weights("glorot")
g_loss = clf.train(x_train, y_train, epochs=10, lr=1e-3)

epoch  1: 1.645
epoch  2: 1.551
epoch  3: 1.527
epoch  4: 1.514
epoch  5: 1.504
epoch  6: 1.497
epoch  7: 1.492
epoch  8: 1.487
epoch  9: 1.484
epoch 10: 1.481


In [8]:
from matplotlib import pyplot as plt
plt.plot(range(0, 10), z_loss, label="zero")
plt.plot(range(0, 10), n_loss, label="normal")
plt.plot(range(0, 10), g_loss, label="glorot")
plt.legend(title="W initialization")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

<Figure size 640x480 with 1 Axes>

In [9]:
print("{:^10s} | {:^10s} | {:^10s} | {:^10s}".format("hidden 1", "hidden 2", "lr", "accuracy"))
print(49 * "-")
for h1 in [400, 200, 100]:
    for h2 in [200, 100, 50]:
        for l in [1e-1, 1e-2, 1e-3]:
            clf = Classifier(hidden_dims=(h1, h2))
            clf.initialize_weights('glorot')
            clf.train(x_train, y_train, epochs=10, lr=l, verbose=False)
            loss, acc = clf.test(x_valid, y_valid)
            print("{:^10d} | {:^10d} | {:^10g} | {:^10.1%}".format(h1, h2, l, acc))

 hidden 1  |  hidden 2  |     lr     |  accuracy 
-------------------------------------------------
   400     |    200     |    0.1     |    9.9%   
   400     |    200     |    0.01    |   98.0%   
   400     |    200     |   0.001    |   97.6%   
   400     |    100     |    0.1     |    9.9%   
   400     |    100     |    0.01    |   97.6%   
   400     |    100     |   0.001    |   97.6%   
   400     |     50     |    0.1     |    9.9%   
   400     |     50     |    0.01    |   97.8%   
   400     |     50     |   0.001    |   97.5%   
   200     |    200     |    0.1     |    9.9%   
   200     |    200     |    0.01    |   97.5%   
   200     |    200     |   0.001    |   97.2%   
   200     |    100     |    0.1     |    9.9%   
   200     |    100     |    0.01    |   97.8%   
   200     |    100     |   0.001    |   97.3%   
   200     |     50     |    0.1     |    9.9%   
   200     |     50     |    0.01    |   97.6%   
   200     |     50     |   0.001    |   97.5%   
